<a href="https://colab.research.google.com/github/theogbrand/learning_llm_handbook/blob/main/notebooks/Week1_LLM_Fundamentals/Week1_LLM_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1: LLM Fundamentals and Development Environment Setup
Resource Required: Google Colab (T4) GPU - free

💡 NOTE: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4.



In [1]:
!pip install transformers>=4.40.1 accelerate>=0.27.2

## Notebook 1.1: LLM Tools and Libraries
- Import SEA-LION using Hugging Face, implement different decoding strategies
- Run inference on Colab T4 GPU and compare with local Ollama deployment
- Explore PyTorch basics for LLM implementation

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "aisingapore/llama3.1-8b-cpt-sea-lionv3-instruct-gguf",
    device_map="cuda", # requires GPU
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("aisingapore/llama3.1-8b-cpt-sea-lionv3-instruct-gguf")

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

messages = [
    {"role": "user", "content": "What is the nature of reality?"}
]

output = generator(messages)
print(output[0]["generated_text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 The nature of reality is a profound philosophical question that has been debated for centuries. It encompasses various interpretations and theories across different philosophical traditions. In Western philosophy, the nature of reality has been explored through metaphysics, which examines the fundamental nature of being and the world.


One perspective is that reality is material and can be understood through empirical observation and scientific inquiry. This view is often associated with physicalism or materialism, which posits that everything that exists is physical or material in nature.


Another perspective is idealism, which suggests that reality is fundamentally mental or spiritual. This view is associated with philosophers like George Berkeley, who argued that objects only exist as perceptions in the minds of observers.


Dualism, most famously proposed by René Descartes, posits that reality consists of two fundamental substances: the physical (res extensa) and the non-physica

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("gpt2-xl")
tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")

tokenizer.pad_token = tokenizer.eos_token # Ignore this for now, we will learn what these are down the line
model.config.pad_token_id = model.config.eos_token_id # Ignore this for now, we will learn what these are down the line

prompt = "What is the nature of reality? The nature of reality is: "
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=100,
    add_special_tokens=True
)

# Generate text
gen_tokens = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    do_sample=True,
    temperature=0.9,
    max_length=100,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# Decode and print the generated text
gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
print(gen_text)

In [ ]:
# Show MinGPT and how it got here. Before RLHF, Instruction Tuning existed, we ONLY had sentence completion

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407) 

In [ ]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2-xl'
device = 'cuda'

if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

In [ ]:
def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [ ]:
generate(prompt='What is the nature of reality? The nature of reality is: ', num_samples=10, steps=20)

## Notebook 1.2: Tokenizers Implementation
- Compare SentencePiece, TikToken, and BPE on various text samples
- Implement a basic BPE tokenizer from scratch (Karpathy Tokenizer video)
- Experiment with vocabulary extension techniques

In [ ]:
# Tokenizer implementation code

## Notebook 1.3: Embeddings Workshop
- Implement contrastive learning with CLIP for image-text pairs
- Create and visualize Word2Vec and contextual embeddings
- Evaluate embeddings using MTEB benchmark tasks
- Compare embedding performance across multiple languages

In [ ]:
# Embeddings implementation code